In [ ]:
import pandas as pd
import torch
from transformers import *
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#Checking if the setup to use GPU instead of cpu is ready.
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

2.3.1+cu121
True
0
1
Tesla T4


**DATA**

In [ ]:
#loading the pre-filtered ontology for level_3  research method and data analysis method.
ontology_df = pd.read_csv('ontology.csv')

#loading the pre-filtered synonyms for level_3  research method and data analysis method.

synonyms_df = pd.read_csv('synonyms.csv')

In [ ]:
#Check general info of the ontology
ontology_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899 entries, 0 to 898
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ent_id      899 non-null    object 
 1   definition  899 non-null    object 
 2   label       898 non-null    object 
 3   parent      899 non-null    object 
 4   level       899 non-null    float64
 5   related     166 non-null    object 
 6   level_1     899 non-null    object 
 7   level_2     899 non-null    object 
 8   level_3     899 non-null    object 
 9   level_4     897 non-null    object 
 10  level_5     893 non-null    object 
 11  level_6     845 non-null    object 
 12  level_7     661 non-null    object 
 13  level_8     462 non-null    object 
 14  level_9     217 non-null    object 
 15  level_10    77 non-null     object 
dtypes: float64(1), object(15)
memory usage: 112.5+ KB


In [ ]:
ontology_df["ent_id"].nunique()

899

In [ ]:
ontology_df["parent"].nunique()

220

In [ ]:
#Check general info of the synonyms
synonyms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31806 entries, 0 to 31805
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   synonym  31806 non-null  object
 1   ent_id   31806 non-null  object
dtypes: object(2)
memory usage: 497.1+ KB


In [ ]:
synonyms_df["synonym"].nunique()

31770

In [ ]:
synonyms_df["ent_id"].nunique()

491

In [ ]:
#print('Before ', synonyms_df.shape)
#synonyms_df.drop_duplicates(inplace=True)
#print('After: ', synonyms_df.shape)
#resetting index to avoid issues
#synonyms_df.reset_index(drop=True, inplace=True)

In [ ]:
data_synonyms = synonyms_df.copy() # Create a copy of the original dataframe

In [ ]:
data_ontology = ontology_df.copy() # Create a copy of the original dataframe

In [ ]:
data_ontology = data_ontology[['parent','ent_id']]

In [ ]:
data_ontology['text'] = data_ontology['ent_id']

In [ ]:
data_synonyms['text'] = data_synonyms['synonym']

**Converting the text in the dataframes to list.**

In [ ]:
data_synonyms = data_synonyms["text"].to_list()
data_ontology = data_ontology["text"].to_list()

**SciBERT  Approach**

In [ ]:
# Assuming 'data_clean' is a list of lists where each inner list contains multiple text elements
#Since it is not a list I can just access the content of the row in the datafrme
data_synonym_bert =  [['[CLS] ' + ' '+ synonyms_df['synonym'].iloc[index] + ' [SEP]'] for index, inner_list in synonyms_df.iterrows()]
data_ontology_bert =  [['[CLS] ' + ' '+ ontology_df['ent_id'].iloc[index] + ' [SEP]'] for index, inner_list in ontology_df.iterrows()]

In [ ]:
# Examine the first list entry
print(data_synonym_bert[0])
print(data_ontology_bert[0])

['[CLS]  QDA Miner Lite system [SEP]']
['[CLS]  research method [SEP]']


In [ ]:
#Apply BERT-tokenization
scibert_tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b1bfb0bcaf9ab193ff3ad01e87732fc1/config.json
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.42.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b

In [ ]:
#loading the pre-trained  model
model_scibert = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b1bfb0bcaf9ab193ff3ad01e87732fc1/config.json
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.42.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b1bfb0bcaf9ab193ff3ad01e87732fc1/pytorch_model.bin
Some weights of the model c

In [ ]:
# Tokenize each text
tokenized_synonym_bert = [scibert_tokenizer.tokenize(text[0]) for text in data_synonym_bert]
tokenized_ontology_bert = [scibert_tokenizer.tokenize(text[0]) for text in data_ontology_bert]



In [ ]:
# Examine the first list entry
print(tokenized_synonym_bert[0])
print(tokenized_ontology_bert[0])

['[CLS]', 'qd', '##a', 'miner', 'lit', '##e', 'system', '[SEP]']
['[CLS]', 'research', 'method', '[SEP]']


In [ ]:
# Calculate the lengths of tokenized texts
text_lengths_synonym_bert = [len(tokens) for tokens in tokenized_synonym_bert]
text_lengths_ontology_bert = [len(tokens) for tokens in tokenized_ontology_bert]
#determine maximum overall length
text_lengths_bert = text_lengths_ontology_bert + text_lengths_synonym_bert


# Determine the 85th percentile for the maximum sequence length
max_seq_length_synonym_bert = int(np.percentile(text_lengths_synonym_bert, 85))
print("Maximum sequence length synonyms:", max_seq_length_synonym_bert)

# Determine the 85th percentile for the maximum sequence length
max_seq_length_ontology_bert = int(np.percentile(text_lengths_ontology_bert, 85))
print("Maximum sequence length Ontology:", max_seq_length_ontology_bert)

# Determine the 85th percentile for the maximum sequence length
max_seq_length_bert= int(np.percentile(text_lengths_bert, 85))
print("Maximum sequence length overall:", max_seq_length_bert)

Maximum sequence length synonyms: 8
Maximum sequence length Ontology: 7
Maximum sequence length overall: 8


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the GPU
model_scibert.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31090, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
from utils.embeddings_utils import get_embeddings

embeddings_ontology_scibert = get_embeddings(terms=ontology_df['ent_id'],
                                             model_type = 'bert',
                                             model=model_scibert,
                                             tokenizer=scibert_tokenizer,max_seq_length =max_seq_length_bert)


In [ ]:
embeddings_synonyms_scibert = get_embeddings(terms=synonyms_df['synonym'],
                                             model_type = 'bert',
                                             model=model_scibert,
                                             tokenizer=scibert_tokenizer,max_seq_length =max_seq_length_bert)

In [ ]:
from utils.model_utils import save_embeddings_to_pickle

save_embeddings_to_pickle(embeddings_ontology_scibert, 'embeddings_ontology_scibert.pkl')


In [ ]:
save_embeddings_to_pickle(embeddings_synonyms_scibert, 'embeddings_synonyms_scibert.pkl')

**Fasttext** Aproach

In [ ]:
import requests
import zipfile
import os

def download_and_extract_fasttext(url, download_path, extract_path):
    """
    Downloads and extracts a zip file from a given URL.

    Args:
        url (str): The URL of the zip file to download.
        download_path (str): The path where the zip file will be saved.
        extract_path (str): The directory where the contents will be extracted.

    Returns:
        None
    """
    # Download the file
    response = requests.get(url)
    with open(download_path, 'wb') as file:
        file.write(response.content)

    # Extract the file
    with zipfile.ZipFile(download_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # Remove the zip file after extraction
    os.remove(download_path)

# URL of the FastText model
url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M-subword.vec.zip'

# Paths
download_path = 'wiki-news-300d-1M-subword.vec.zip'
extract_path = 'fasttext_vectors'

# Create the extract_path directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Download and extract the FastText model
download_and_extract_fasttext(url, download_path, extract_path)

print(f"FastText model downloaded and extracted to '{extract_path}'")


FastText model downloaded and extracted to 'fasttext_vectors'


In [ ]:
from gensim.models import KeyedVectors
import re
from utils.embeddings_utils import get_embeddings

In [ ]:
# Load pre-trained FastText vectors
fasttext_model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M-subword.vec')

In [ ]:
embeddings_ontology_fasttext = get_embeddings(terms=ontology_df['ent_id'],
                                             model_type = 'fasttext',
                                             model=fasttext_model)

In [ ]:
embeddings_synonyms_fasttextc = get_embeddings(terms=synonyms_df['synonym'],
                                             model_type = 'fasttext',
                                             model=fasttext_model)

In [ ]:
save_embeddings_to_pickle(embeddings_ontology_fasttext, 'embeddings_ontology_fasttext.pkl')

In [ ]:
save_embeddings_to_pickle(embeddings_synonyms_fasttext, 'embeddings_synonyms_fasttext.pkl')

**Sheilla** Aproach (Word2vec?)

In [ ]:
data_synonyms_list = data_synonyms.copy()
data_ontology_list = data_ontology.copy()

In [ ]:
# Combine both lists to create a corpus
combined_data = data_synonyms_list + data_ontology_list

# Tokenize the text (simple whitespace tokenization)
tokenized_data = [text.split() for text in combined_data]

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [ ]:
# Train the Word2Vec model
model_word2vec = Word2Vec(tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
# Get vectors for each word in data_synonyms_list and data_ontology_list
embeddings_synonyms_word2vec = {word: model_word2vec.wv[word] for text in data_synonyms_list for word in text.split()}
embeddings_ontology_word2vec = {word: model_word2vec.wv[word] for text in data_ontology_list for word in text.split()}

In [ ]:
embeddings_ontology_word2vec = get_embeddings(terms=ontology_df['ent_id'],
                                             model_type = 'word2vec',
                                             model=model_word2vec)

In [ ]:
embeddings_synonyms_word2vec = get_embeddings(terms=synonyms_df['synonym'],
                                             model_type = 'word2vec',
                                             model=model_word2vec)

In [ ]:
save_embeddings_to_pickle(embeddings_ontology_word2vec, 'embeddings_ontology_word2vec.pkl')

In [ ]:
save_embeddings_to_pickle(embeddings_synonyms_word2vec, 'embeddings_synonyms_word2vec.pkl')